In [77]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
from scipy import signal
import IPython.display as ipd

In [78]:
def sinusoid(freq=440.0, dur=1.0, srate=44100.0, amp=1.0, phase = 0.0): 
    t = np.linspace(0,dur,int(srate*dur))
    data = amp * np.sin(2*np.pi*freq *t+phase)
    return data

In [79]:
def generate_tone(freqs, dur=1.0, srate=44100.0):
    data = np.zeros(int(dur*srate))
    for freq in freqs:
        data += sinusoid(freq, dur, srate, np.random.rand(), np.random.rand()*2*np.pi)
    return data

In [80]:
#testing
freqs = np.random.rand(3)*700+200
data = generate_tone(freqs)
ipd.Audio(data, rate=44100)

In [81]:
def analyze_tone_dft(freqs, data, srate=44100.0):
    N = len(data)

    #predetermine the linearly spaced frequencies (bins)
    #that will be used in the FFT
    freqs_dft = np.linspace(0, srate, N)
    #compute the DFT of the signal
    dft = np.fft.fft(data)

    #extract the amplitudes and phases of the 3 frequencies
    amplitudes = np.zeros(3)
    phases = np.zeros(3)
    for i, freq in enumerate(freqs):
        #finds closest DFT bin to selected frequency using above predetermined bins
        idx = np.argmin(np.abs(freqs_dft-freq))
        #extract amplitude and phase of the DFT at that bin
        amplitudes[i] = np.abs(dft[idx])/N
        phases[i] = np.angle(dft[idx])
    return amplitudes, phases

In [82]:
#testing
freqs = np.random.rand(3)*700+200
data = generate_tone(freqs)
amplitudes, phases = analyze_tone_dft(freqs, data)
print(amplitudes)
print(phases)

[0.25290926 0.31970257 0.10660203]
[ 2.19159708  3.02680081 -2.76879945]


In [83]:
def evaluate_analysis(amp_orig, amp_est):
    return np.mean((np.array(amp_orig) - np.array(amp_est))**2)

In [84]:
def generate_tone_with_amps(freqs, dur=1.0, srate=44100.0):
    data = np.zeros(int(dur*srate))
    amps = []
    for freq in freqs:
        amp = np.random.rand()
        amps.append(amp)
        data += sinusoid(freq, dur, srate, amp, np.random.rand()*2*np.pi)
    return data, amps

In [88]:
#testing
freqs = np.random.rand(3)*700+200
data, amps = generate_tone_with_amps(freqs)
amplitudes, phases = analyze_tone_dft(freqs, data)
print(evaluate_analysis(amps, amplitudes))

0.18016277555351723


In [86]:

def analyze_tone_fixed_frequencies(freqs, data, srate=44100.0):
    N = len(data)
    T = 1.0/srate
    #create time vector
    t = np.linspace(0.0, N*T, N, endpoint=False)
    amplitudes = []
    phases = []
    for freq in freqs:
        #compute the DFT of the signal using specified frequencies as basis vectors
        y = np.sum(data * np.exp(-2j * np.pi * freq * t)) / N
        amplitudes.append(np.abs(y))
        phases.append(np.angle(y))
    return amplitudes, phases

In [87]:
#testing
freqs = np.random.rand(3)*700+200
data, amps = generate_tone_with_amps(freqs)
amplitudes, phases = analyze_tone_fixed_frequencies(freqs, data)
print(evaluate_analysis(amps, amplitudes))


0.09728915625127034
